<a href="https://colab.research.google.com/github/Noreen999/Digital-Image-Processing/blob/main/DIP_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#DIP project

##Install libraries

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install opencv-python pillow
!pip install numpy
!pip install matplotlib
!pip install torch



## Import Libraries

In [4]:
import cv2
import numpy as np
from IPython.display import display, Image
import os


# Step 1. Reading and preparing our frame

In [5]:
!pip install opencv-python pillow


In [6]:
# Function to process the video and track motion using basic image processing techniques
def motion_detector_video(video_path, output_folder):
    cap = cv2.VideoCapture(video_path)
    previous_frame = None
    frame_count = 0

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert frame to grayscale
        prepared_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        prepared_frame = cv2.GaussianBlur(src=prepared_frame, ksize=(5, 5), sigmaX=0)

        if previous_frame is None:
            previous_frame = prepared_frame
            continue

        # Calculate difference between current frame and previous frame
        diff_frame = cv2.absdiff(src1=previous_frame, src2=prepared_frame)
        previous_frame = prepared_frame

        # Dilate the difference image
        kernel = np.ones((5, 5))
        diff_frame = cv2.dilate(diff_frame, kernel, 1)

        # Threshold the difference image
        thresh_frame = cv2.threshold(src=diff_frame, thresh=20, maxval=255, type=cv2.THRESH_BINARY)[1]

        # Find contours in the thresholded image
        contours, _ = cv2.findContours(image=thresh_frame, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_SIMPLE)

        # Draw rectangles around the detected contours
        for contour in contours:
            if cv2.contourArea(contour) < 50:
                continue
            (x, y, w, h) = cv2.boundingRect(contour)
            cv2.rectangle(img=frame, pt1=(x, y), pt2=(x + w, y + h), color=(0, 255, 0), thickness=2)

        # Save the frame as an image file
        output_path = os.path.join(output_folder, f"frame_{frame_count}.png")
        cv2.imwrite(output_path, frame)
        frame_count += 1

    cap.release()
    cv2.destroyAllWindows()


##Run

## generate new video with featues

In [8]:
import cv2
import os

def create_video_from_frames(input_folder, output_folder, output_video_name):
    # Ensure the output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Make sure the output video name has a valid extension
    if not output_video_name.lower().endswith(('.mp4', '.avi')):
        print("Error: Output video name must have a .mp4 or .avi extension")
        return

    output_video_path = os.path.join(output_folder, output_video_name)

    # Get all the frames from the input folder
    frames = [f for f in os.listdir(input_folder) if f.endswith('.png')]
    frames.sort(key=lambda x: int(x.split('_')[1].split('.')[0]))  # Sort by frame number

    if not frames:
        print("Error: No frames found in the input folder")
        return

    # Create a VideoWriter object
    first_frame_path = os.path.join(input_folder, frames[0])
    first_frame = cv2.imread(first_frame_path)
    if first_frame is None:
        print("Error: Unable to read the first frame")
        return

    frame_size = (first_frame.shape[1], first_frame.shape[0])
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # or use 'XVID' for .avi format
    video_writer = cv2.VideoWriter(output_video_path, fourcc, 30.0, frame_size)  # 30 fps

    for frame_name in frames:
        frame_path = os.path.join(input_folder, frame_name)
        frame = cv2.imread(frame_path)
        if frame is None:
            print(f"Warning: Unable to read frame {frame_name}")
            continue
        video_writer.write(frame)  # Write the frame to the video

    video_writer.release()
    print(f"Video saved to {output_video_path}")

# Example paths
input_folder = '/content/frames'  # Directory containing the frames
output_folder = '/content'  # Directory where the video will be saved
output_video_name = 'new_video.mp4'  # Ensure it has a valid extension

create_video_from_frames(input_folder, output_folder, output_video_name)


Video saved to /content/new_video.mp4


# Apply on Project

In [18]:
# Example video path and output folder
video_path = '/content/vedio.mp4'
output_folder = '/content/frames'
motion_detector_video(video_path, output_folder)